In [15]:
import os
import re
import pandas as pd
import pickle

In [2]:
filepath = os.getcwd() + '/data/'

In [3]:
df = pd.read_csv(filepath + 'all_sources_metadata_2020-03-13.csv')

In [4]:
df

,sha,source_x,title,doi,pmcid,pubmed_id,license,abstract,publish_time,authors,journal,Microsoft Academic Paper ID,WHO #Covidence,has_full_text
0,c630ebcdf30652f0422c3ec12a00b50241dc9bd9,CZI,Angiotensin-converting enzyme 2 (ACE2) as a SA...,10.1007/s00134-020-05985-9,NaN,32125455.0,cc-by-nc,NaN,2020,"Zhang, Haibo; Penninger, Josef M.; Li, Yimin; ...",Intensive Care Med,2.002765e+09,#3252,True
1,53eccda7977a31e3d0f565c884da036b1e85438e,CZI,Comparative genetic analysis of the novel coro...,10.1038/s41421-020-0147-1,NaN,NaN,cc-by,NaN,2020,"Cao, Yanan; Li, Lin; Feng, Zhimin; Wan, Shengq...",Cell Discovery,3.003431e+09,#1861,True
2,210a892deb1c61577f6fba58505fd65356ce6636,CZI,Incubation Period and Other Epidemiological Ch...,10.3390/jcm9020538,NaN,NaN,cc-by,The geographic spread of 2019 novel coronaviru...,2020,"Linton, M. Natalie; Kobayashi, Tetsuro; Yang, ...",Journal of Clinical Medicine,3.006065e+09,#1043,True
3,e3b40cc8e0e137c416b4a2273a4dca94ae8178cc,CZI,Characteristics of and Public Health Responses...,10.3390/jcm9020575,NaN,32093211.0,cc-by,"In December 2019, cases of unidentified pneumo...",2020,"Deng, Sheng-Qun; Peng, Hong-Juan",J Clin Med,1.776631e+08,#1999,True
4,92c2c9839304b4f2bc1276d41b1aa885d8b364fd,CZI,Imaging changes in severe COVID-19 pneumonia,10.1007/s00134-020-05976-w,NaN,32125453.0,cc-by-nc,NaN,2020,"Zhang, Wei",Intensive Care Med,3.006643e+09,#3242,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29495,c42a617a00afe6a36bde0a8e3638e0f55bfee4f7,medrxiv,Prediction of COVID-19 Spreading Profiles in S...,doi.org/10.1101/2020.03.08.20032847,NaN,NaN,See https://www.medrxiv.org/submit-a-manuscript,<jats:p>This work applies a data-driven coding...,2020-03-10,"['Choujun Zhan', 'Chi K. Tse', 'Zhikang Lai', ...",NaN,NaN,NaN,True
29496,c4ce14ce42fa4360dfe3515ec9d1584847381c27,medrxiv,A deterministic epidemic model for the emergen...,doi.org/10.1101/2020.03.08.20032854,NaN,NaN,See https://www.medrxiv.org/submit-a-manuscript,<jats:p>Coronavirus disease (COVID-19) broke o...,2020-03-10,"['Meng Wang', 'Jingtao Qi']",NaN,NaN,NaN,True
29497,c41e09a32be90c84cea0616bb1c726aecba721e0,medrxiv,Data-driven discovery of clinical routes for s...,doi.org/10.1101/2020.03.09.20032219,NaN,NaN,See https://www.medrxiv.org/submit-a-manuscript,<jats:p>The outbreak of COVID-19 epidemic has ...,2020-03-10,"['Hui Yu', 'Jianbo Shao', 'Yuqi Guo', 'Yun Xia...",NaN,NaN,NaN,False
29498,ca88735399ff43d0e673876200655099f06f5567,medrxiv,Ascertainment rate of novel coronavirus diseas...,doi.org/10.1101/2020.03.09.20033183,NaN,NaN,See https://www.medrxiv.org/submit-a-manuscript,<jats:p>We analyzed the epidemiological datase...,2020-03-10,"['Ryosuke Omori', 'Kenji Mizumoto', 'Hiroshi N...",NaN,NaN,NaN,True


In [5]:
sources = ['biorxiv_medrxiv.tar.gz','pmc_custom_license.tar.gz','noncomm_use_subset.tar.gz','comm_use_subset.tar.gz']

In [6]:
import tarfile
import json
import numpy as np

In [7]:
def pre_proc(text_array):
    # remove inline references and citations and returns a plain_text of the entire paper (abstract + body)
    remove_spans = lambda i,j: i['text'].replace(i['text'][int(j['start']):int(j['end'])],' '*(int(j['end'])-int(j['start'])))
    plain_text = ""
    for i in text_array:
        for j in i['cite_spans']:
            i['text'] = remove_spans(i,j)
        for j in i['ref_spans']:
            i['text'] = remove_spans(i,j)
    for i in text_array:
        plain_text += i['text']
    # remove URL
    plain_text = re.sub(r'^(https|http|ftp)?:\/\/.*[\r\n]*', '', plain_text, flags=re.MULTILINE)
    # pattern
    pattern1 = r'. CC-BY-NC-ND 4.0 International license is made available under a'
    pattern2 = r'The copyright holder for this preprint (which was not peer-reviewed) is the author/funder.'
    pattern3 = r'It .'
    pattern4 = r'doi: bioRxiv preprint'
    pattern5 = r'((ftp|https?):((//)|(\\\\))+[\w\d:#@%/;$()~_?\+-=\\\.&]*)'
    
    plain_text = re.sub(pattern5,'<URI>',plain_text, flags=re.MULTILINE|re.UNICODE)
    
    def remove_pattern(pattern,text):
        i = text.find(pattern)
        while i != -1:
            j = i+pattern.__len__()
            text = text.replace(text[i:j], '')
            i = text.find(pattern)
        return text
    
    plain_text = remove_pattern(pattern1, plain_text)
    plain_text = remove_pattern(pattern2, plain_text)
    plain_text = remove_pattern(pattern3, plain_text)
    plain_text = remove_pattern(pattern4, plain_text)
        
    return plain_text

In [9]:
data = []

In [10]:
for s in sources:
    tar = tarfile.open(filepath+s,'r:gz')
    for member in tar.getmembers():
        f = tar.extractfile(member)
        if f is not None:
            content = f.read()
            paper = json.loads(content)
            body_text = [t for t in paper['abstract']+paper['body_text']]
            data.append(pre_proc(body_text))
            body_text = None

In [11]:
data.__len__()

13202

In [12]:
paper['paper_id']

'a70e7c4d8ee484ce956e91c8700d0c9310bbdbbc'

In [13]:
paper['metadata']['title']

'Estimated effectiveness of symptom and risk screening to prevent the spread of COVID-19'

In [16]:
with open('data/paper_data.pkl','wb') as f:
    pickle.dump(data, f)